In [4]:
import time
from datetime import datetime, timedelta
from ib_insync import *
import numpy as np
import pandas as pd
import os
from ib_insync import *


# Install ib_insync if you haven't already: pip install ib_insync

def calculate_pivot_points(high, low, close):
    pivot = (high + low + close) / 3
    s3 = pivot - 2 * (high - low)
    r3 = pivot + 2 * (high - low)
    s4 = low - 2 * (high - pivot)
    r4 = high + 2 * (pivot - low)
    return pivot, s3, r3, s4, r4

def alert(title, message):
    os.system(f'notify-send "{title}" "{message}"')

def main():
    print("connecting to IBKR API")
    util.startLoop()
    ib = IB()
    ib.connect()

    symbols = ['AAPL', 'MSFT', 'GOOG']  # Add your list of stocks here
    contracts = [Stock(symbol, 'SMART', 'USD') for symbol in symbols]

    bars = {symbol: [] for symbol in symbols}
    prices = {symbol: None for symbol in symbols}
    
    def onBarUpdate(bars, hasNewBar):
        for symbol, bar_list in bars.items():
            if len(bar_list) < 3:
                continue
            
            recent_bar = bar_list[-1]
            high = max(bar.high for bar in bar_list)
            low = min(bar.low for bar in bar_list)
            close = bar_list[-2].close  # Use the close of the previous bar for calculations
            
            pivot, s3, r3, s4, r4 = calculate_pivot_points(high, low, close)
            vwap = np.mean([bar.close for bar in bar_list])
            
            current_price = recent_bar.close
            prices[symbol] = current_price

            if abs(current_price - vwap) < 0.1 or abs(current_price - s3) < 0.1 or abs(current_price - r3) < 0.1 or abs(current_price - s4) < 0.1 or abs(current_price - r4) < 0.1:
                alert(f'Alert for {symbol}', f'Price near critical level: {current_price:.2f}')
    
    for contract in contracts:
        bars.update({contract.symbol: ib.reqHistoricalData(
            contract,
            endDateTime='',
            durationStr='1 D',
            barSizeSetting='1 min',
            whatToShow='MIDPOINT',
            useRTH=True,
            formatDate=1,
            keepUpToDate=True,
            timeout=10
        )})
    
    for bar in bars.values():
        bar.updateEvent += onBarUpdate
    
    try:
        ib.run()
    except KeyboardInterrupt:
        print("Stopping script")
    finally:
        ib.disconnect()

if __name__ == "__main__":
    main()


connecting to IBKR API


In [5]:
import yfinance as yf
import numpy as np
import pandas as pd
import os
import time
from datetime import datetime, timedelta

def calculate_pivot_points(high, low, close):
    pivot = (high + low + close) / 3
    s3 = pivot - 2 * (high - low)
    r3 = pivot + 2 * (high - low)
    s4 = low - 2 * (high - pivot)
    r4 = high + 2 * (pivot - low)
    return pivot, s3, r3, s4, r4

def alert(title, message):
    os.system(f'notify-send "{title}" "{message}"')

def fetch_live_data(symbol):
    ticker = yf.Ticker(symbol)
    hist = ticker.history(period="1d", interval="1m")
    return hist

def check_conditions(symbol, data):
    high = data['High'].max()
    low = data['Low'].min()
    close = data['Close'][-2]
    current_price = data['Close'][-1]
    
    pivot, s3, r3, s4, r4 = calculate_pivot_points(high, low, close)
    vwap = np.mean(data['Close'])

    if abs(current_price - vwap) < 0.1 or abs(current_price - s3) < 0.1 or abs(current_price - r3) < 0.1 or abs(current_price - s4) < 0.1 or abs(current_price - r4) < 0.1:
        alert(f'Alert for {symbol}', f'Price near critical level: {current_price:.2f}')
        return True
    return False

def main():
    symbols = ['AAPL', 'MSFT', 'GOOG','NVDA', 'AMD', 'TSLA','MU','MARA']  # Add your list of stocks here
    
    while True:
        for symbol in symbols:
            data = fetch_live_data(symbol)
            if not data.empty:
                condition_met = check_conditions(symbol, data)
                if condition_met:
                    print(f'Condition met for {symbol} at {datetime.now()}')
        time.sleep(30)

if __name__ == "__main__":
    main()


Condition met for GOOG at 2024-06-24 16:57:43.776849
Condition met for AMD at 2024-06-24 16:57:43.989364
Condition met for MARA at 2024-06-24 16:57:44.413277
Condition met for MSFT at 2024-06-24 16:58:14.623520
Condition met for GOOG at 2024-06-24 16:58:14.736839
Condition met for AMD at 2024-06-24 16:58:14.948125
Condition met for MARA at 2024-06-24 16:58:15.273465
Condition met for GOOG at 2024-06-24 16:58:45.586804
Condition met for MARA at 2024-06-24 16:58:46.220683
Condition met for GOOG at 2024-06-24 16:59:16.541450
Condition met for MARA at 2024-06-24 16:59:17.065639
Condition met for GOOG at 2024-06-24 16:59:48.024444
Condition met for MARA at 2024-06-24 16:59:48.706499


KeyboardInterrupt: 

In [6]:
import yfinance as yf
import numpy as np
import pandas as pd
import os
import time
from datetime import datetime, timedelta

def calculate_pivot_points(high, low, close):
    pivot = (high + low + close) / 3
    s3 = pivot - 2 * (high - low)
    r3 = pivot + 2 * (high - low)
    s4 = low - 2 * (high - pivot)
    r4 = high + 2 * (pivot - low)
    return pivot, s3, r3, s4, r4

def calculate_vwap(data):
    typical_price = (data['Close'] + data['High'] + data['Low']) / 3
    vwap = (typical_price * data['Volume']).cumsum() / data['Volume'].cumsum()
    return vwap.iloc[-1]

def alert(title, message):
    os.system(f'notify-send "{title}" "{message}"')

def fetch_live_data(symbol):
    ticker = yf.Ticker(symbol)
    hist = ticker.history(period="1d", interval="1m")
    return hist

def check_conditions(symbol, data):
    high = data['High'].max()
    low = data['Low'].min()
    close = data['Close'][-2]
    current_price = data['Close'][-1]
    
    pivot, s3, r3, s4, r4 = calculate_pivot_points(high, low, close)
    vwap = calculate_vwap(data)

    if current_price > s3 and current_price - s3 < 0.1:
        alert(f'Alert for {symbol}', f'Price approaching S3 from above: {current_price:.2f}')
        return True
    if current_price > s4 and current_price - s4 < 0.1:
        alert(f'Alert for {symbol}', f'Price approaching S4 from above: {current_price:.2f}')
        return True
    if current_price < r3 and r3 - current_price < 0.1:
        alert(f'Alert for {symbol}', f'Price approaching R3 from below: {current_price:.2f}')
        return True
    if current_price < r4 and r4 - current_price < 0.1:
        alert(f'Alert for {symbol}', f'Price approaching R4 from below: {current_price:.2f}')
        return True
    if abs(current_price - vwap) < 0.1:
        alert(f'Alert for {symbol}', f'Price approaching VWAP: {current_price:.2f}')
        return True
    return False

def main():
    symbols = ['AAPL', 'MSFT', 'GOOG','NVDA', 'AMD', 'TSLA','MU','MARA']   # Add your list of stocks here
    
    while True:
        for symbol in symbols:
            data = fetch_live_data(symbol)
            if not data.empty:
                condition_met = check_conditions(symbol, data)
                if condition_met:
                    print(f'Condition met for {symbol} at {datetime.now()}')
        time.sleep(30)

if __name__ == "__main__":
    main()


Condition met for GOOG at 2024-06-24 17:06:25.018393
Condition met for GOOG at 2024-06-24 17:06:55.952487


AAPL: No price data found, symbol may be delisted (period=1d)


Condition met for GOOG at 2024-06-24 17:07:36.979578
Condition met for MU at 2024-06-24 17:07:37.436044
Condition met for GOOG at 2024-06-24 17:08:07.900600
Condition met for GOOG at 2024-06-24 17:08:38.877339
Condition met for GOOG at 2024-06-24 17:09:09.784213
Condition met for MSFT at 2024-06-24 17:09:40.635667
Condition met for GOOG at 2024-06-24 17:10:11.697751
Condition met for GOOG at 2024-06-24 17:10:42.612027
Condition met for GOOG at 2024-06-24 17:11:13.586337
Condition met for GOOG at 2024-06-24 17:11:44.526193
Condition met for MSFT at 2024-06-24 17:12:15.341572
Condition met for GOOG at 2024-06-24 17:12:15.467381
Condition met for MSFT at 2024-06-24 17:12:46.308771
Condition met for GOOG at 2024-06-24 17:12:46.440471
Condition met for MSFT at 2024-06-24 17:13:17.913738
Condition met for GOOG at 2024-06-24 17:13:18.037402
Condition met for MSFT at 2024-06-24 17:13:48.851806
Condition met for GOOG at 2024-06-24 17:13:48.971620
Condition met for GOOG at 2024-06-24 17:14:19.89

GOOG: No price data found, symbol may be delisted (period=1d)


Condition met for GOOG at 2024-06-24 17:28:57.488352
Condition met for GOOG at 2024-06-24 17:29:28.381948
Condition met for GOOG at 2024-06-24 17:29:59.274166
Condition met for MSFT at 2024-06-24 17:31:00.914711
Condition met for MSFT at 2024-06-24 17:31:31.805660
Condition met for MSFT at 2024-06-24 17:32:02.703841
Condition met for MSFT at 2024-06-24 17:32:33.594413
Condition met for MSFT at 2024-06-24 17:33:04.499660
Condition met for MSFT at 2024-06-24 17:35:08.122574
